# DS 6 Corrigé (Un système de vote : CentraleSupélec 2020)

## Vote par préférence

In [1]:
(* Q1 (/1.5) *) 
let duel i j urne =
    let rec compare = function
      | e::q -> 
        if e = i then 1
        else if e = j then -1
        else compare q
      | [] -> failwith "i et j n'apparaissent pas" in
    List.map compare urne
    |> List.fold_left (+) 0;;

let ex = [[2; 0; 1]; [2; 1; 0]; [1; 2; 0]];;
duel 0 1 ex;;


val duel : 'a -> 'a -> 'a list list -> int = <fun>


val ex : int list list = [[2; 0; 1]; [2; 1; 0]; [1; 2; 0]]


- : int = -1


Q2. (/0.5) Voici le graphe et la matrice d'adjacence associée :

[![](https://mermaid.ink/img/pako:eNpdjrEOwjAQQ38l8txK0DEDU_kCGLOcekcbQZLqSIaq7b8TJFjqyZafJa8YEgssRqV5MvfeRVN1Mm172brNnA-5Q4MgGshzHa3f1iFPEsTBVsukTwcX98qVmSnLlX1OCvug11saUMnptsQBNmuRP9R7qgfCj9o_AA8vZw)](https://mermaid.live/edit#pako:eNpdjrEOwjAQQ38l8txK0DEDU_kCGLOcekcbQZLqSIaq7b8TJFjqyZafJa8YEgssRqV5MvfeRVN1Mm172brNnA-5Q4MgGshzHa3f1iFPEsTBVsukTwcX98qVmSnLlX1OCvug11saUMnptsQBNmuRP9R7qgfCj9o_AA8vZw)

$$\begin{pmatrix}
0 & 2 & 2\\
-2 & 0 & 0\\
-2 & 0 & 0
\end{pmatrix}$$

Q3. (/1) 
- Si le résultat de la comparaison de $i$ avec $j$ est $k$, alors le résultat de la comparaison de $j$ avec $i$ est $-k$, d'où la fait que la matrice d'adjacence soit antisymétrique.  
- Soit $p$ le nombre de votants et $i$, $j$ deux candidats. Le résultat de la comparaison de $i$ et $j$ pour $1$ candidat donne $1$ ou $-1$, et est donc congru à $1$ modulo $2$. Le résultat de la comparaison de  $i$ et $j$ (pour tous les candidats) est donc $p$ modulo $2$, ce qui ne dépend pas de $i$ et $j$. Donc toutes les comparaisons entre candidats ont la même valeur modulo $2$, c'est-à-dire qu'ils ont tous la même parité. 

In [2]:
(* Q4 (/1) *)
let depouillement n u =
  let m = Array.make_matrix n n 0 in
  for i = 0 to n - 1 do
    for j = 0 to n - 1 do
      if i <> j then m.(i).(j) <- duel i j u
    done
  done;
  m;;

depouillement 3 ex

val depouillement : int -> int list list -> int array array = <fun>


- : int array array = [|[|0; -1; -3|]; [|1; 0; -1|]; [|3; 1; 0|]|]


Q5. (/1.5) On considère les votes `[i; j; 0; ...; n - 1]` et `[n - 1; ...; 0; i; j]`, où `0; ...; n - 1` contient tous les entiers de `0` à `n - 1`, sauf `i` et `j`. De même pour `n - 1; ...; 0` mais dans l'ordre décroissant.  
Alors on a bien Mat($U_{i, j, n}$) = $E_{i, j, n}$.

Q6. (/0.5) Mat($U_1 \cup U_2$) = Mat($U_1$) + Mat($U_2$)

Q7. (/2.5) On note $\vert M \vert = \sum_{i, j} \vert m_{i, j} \vert$ la somme des valeurs absolues des éléments d'une matrice $M$.  
Soit $P(n)$ : "si $M$ est une matrice antisymétrique à coefficients pairs telle que $\vert M \vert \leq n$ alors il existe une urne $U$ telle que $M$ = Mat($U$)".  
Montrons $P(n)$ par récurrence sur $n$.  
- Si $\vert M \vert = 0$ alors une urne $U$ vide (avec aucun votant) vérifie $M$ = Mat($U$). Donc $P(0)$ est vraie.
- Supposons $P(n)$. Soit $M$ est une matrice antisymétrique à coefficients pairs telle que $\vert M \vert \leq n + 1$. Soit $i, j$ tel que $M_{i, j} \neq 0$. Quitte à échanger $i$ et $j$, on peut supposer $M_{i, j} > 0$.  
Considérons alors l'urne' $U_{i, j, n}$ définie en Q5. Soit $M' = M -$ Mat($U_{i, j, n}$). $M'$ est une matrice antisymétrique à coefficients pairs et $\vert M' \vert = \vert M \vert - 4 \leq n$. On peut donc appliquer $P(n)$ pour trouver $U$ telle que $M'$ = Mat($U$).  
Alors, d'après Q6, Mat($U \cup U_{i, j, n}$) = Mat($U$) + Mat($U_{i, j, n}$) = $M$. On a bien démontré $P(n + 1)$.

In [3]:
(* Q8 (/2) On utilise les questions précédentes *) 
let mcgarvey m =
  let n = Array.length m in
  let urne = ref [] in
  for i = 0 to n - 1 do
    for j = 0 to n - 1 do
      if m.(i).(j) > 0 then
      let rec aux k =
        if k = n then []
        else if k = i || k = j then aux (k + 1)
        else k::aux (k + 1) in
      let u1 = i::j::aux 0 in (* Q5 *)
      let u2 = (List.rev (aux 0))@[i; j] in
      for k = 0 to m.(i).(j)/2 - 1 do
        urne := u1::u2::!urne
      done
    done
  done;
  !urne;;

  let m_ex =
  [|[|0; 2; -4|];
    [|-2; 0; 2|];
    [|4; -2; 0|]|];;

  mcgarvey m_ex

val mcgarvey : int array array -> int list list = <fun>


val m_ex : int array array = [|[|0; 2; -4|]; [|-2; 0; 2|]; [|4; -2; 0|]|]


- : int list list =
[[2; 0; 1]; [1; 2; 0]; [2; 0; 1]; [1; 2; 0]; [1; 2; 0]; [0; 1; 2]; [0; 1; 2];
 [2; 0; 1]]


Q9. O($n^2(n + q)$)

## Recherche du vainqueur

In [4]:
(* Q11 (/1) *)
let condorcet m =
  let l = ref [] in
  for i = 0 to Array.length m - 1 do
    if Array.for_all ((>) 0) m.(i) 
    then l := i::!l
  done;
  !l

val condorcet : int array array -> int list = <fun>


Q12. (/1) Urne : `[[0; 1; 2]; [1; 2; 0]; [2; 0; 1]]`.  
Graphe :  
[![](https://mermaid.ink/img/pako:eNo1jsEOgyAQRH-FzFkT8cihJ_sF7ZHLRrZKWsBQOBj130uj7Gle5mWyG8ZgGApTpGUWz0F7Ua4TbXvb5S7kybJyf3JfuUMDx9GRNWVk-7caaWbHGqpEQ_Gtof1RvLwYSnw3NoUI9aLPlxtQTuGx-hEqxcxVGiyVh9xlHT9fXDMT)](https://mermaid.live/edit#pako:eNo1jsEOgyAQRH-FzFkT8cihJ_sF7ZHLRrZKWsBQOBj130uj7Gle5mWyG8ZgGApTpGUWz0F7Ua4TbXvb5S7kybJyf3JfuUMDx9GRNWVk-7caaWbHGqpEQ_Gtof1RvLwYSnw3NoUI9aLPlxtQTuGx-hEqxcxVGiyVh9xlHT9fXDMT)

Q13. Le poids d'un chemin n'est pas changé si on passe plusieurs fois par le même sommet. On peut donc se restreindre aux chemins sans boucle.

Q14. Soit $C_1$ un chemin de $i$ à $j$ de poids $I[i, j]$ et $C_2$ un chemin de $j$ à $k$ de poids $I[j, k]$. Alors la concaténation de $C_1$ et $C_2$ est un chemin de $i$ à $k$ de poids $\min(I[i, j], I[j, k])$. Comme $I[i, k]$ est le poids maximum d'un chemin de $i$ à $k$, on a bien $\boxed{\min(I[i, j], I[j, k]) \leq I[i, k]}$.

In [5]:
(* Q15 (/1.5) *)
let intermediaire g =
  let n = Array.length g in
  let d = Array.map Array.copy g in (* copie de g *)
  for k = 0 to n - 1 do
    for i = 0 to n - 1 do
      for j = 0 to n - 1 do
        d.(i).(j) <- max d.(i).(j) (min d.(i).(k) d.(k).(j))
      done
    done
  done;
  d

val intermediaire : 'a array array -> 'a array array = <fun>


Q16. (/0.5) O($n^3$)

Q17. (/1.5) On pourrait adapter Dijkstra en utilisant une file de priorité max avec $\min d.(u) w.(u).(v)$ comme priorité.  
Dans sa version classique, Dijkstra ne fonctionne pas avec des poids négatifs. Cependant, dans cette nouvelle version, l'algorithme fonctionne même avec des poids négatifs (reprendre la démonstration du cours pour s'en convaincre). On pourrait donc utiliser $n$ fois (depuis chaque sommet) cette version modifiée de Dijkstra.  
La complexité de Dijkstra (avec une file de priorité implémenté par un tas ou un ABR équilibré, comme dans le cours) est en O($p\log(n)$) où $p$ est le nombre d'arêtes et $n$ le nombre de sommets. Ici, $p = \binom{2}{n}$ = O($n^2$) vu que le graphe de préférence est complet. Donc utiliser $n$ fois Dijsktra est en O($n^3 \log(n)$), ce qui est moins intéressant que Floyd-Warshall.

Q18. 
- $S[i, j] = I[i, j] - I[j, i] = -(I[j, i] - I[i, j]) = -S[j, i]$  
- $I[i, j]$ et $I[j, i]$ correspondent à deux poids du graphe de préférence. D'après Q3, ils sont donc de même parité. Donc $S[i, j] = I[i, j] - I[j, i]$ est pair (la somme de deux entiers de même parité est toujours paire).

In [6]:
(* Q19 *)
let graphe_schulze m =
  let n = Array.length m in
  let g = Array.make_matrix n n 0 in
  for i = 0 to n - 1 do
    for j = 0 to n - 1 do
        g.(i).(j) <- m.(i).(j) - m.(j).(i)
    done
  done;
  g

val graphe_schulze : int array array -> int array array = <fun>


In [7]:
(* Q20 *)
let schulze n u =
  depouillement n u
  |> intermediaire
  |> graphe_schulze
  |> condorcet

val schulze : int -> int list list -> int list = <fun>


Q 21. 
- `duel` est en O($np$)
- `depouillement` appelle $n^2$ fois `duel`, donc est en O($n^3p$)
- `graphe_schulze` est en O($n^2$)
- `condorcet` est en O($n^2$)
- 
  Conclusion : `schulze` est en $\boxed{\text{O}(n^3p)}$